<a href="https://colab.research.google.com/github/CharlesCarpes/UsarAPIBetfair/blob/main/Pega_jogos_BetFair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install betfairlightweight

In [ ]:
!pip install python-dotenv

In [2]:
import datetime
from datetime import datetime, timedelta, timezone
import pandas as pd
import betfairlightweight
import time
import dotenv
import os

In [ ]:
#Aqui você vai colocar seu dados pessoas!
#cuidado para não compartilhar seu código com esses dados visíveis

email = "seu_email_betfair@gmail.com"
senha = "sua_senha_da_betfair"
app_key = "sua_key_gerada_no_video_anterior"

In [7]:
#Esses são os comandos de importação para usar no COLAB
from google.colab import userdata
email = userdata.get('email')
senha = userdata.get('senha')
app_key = userdata.get('app_key')

In [ ]:
# Carrega as variáveis do arquivo .env
# Só funciona se estiver usando o Jupyter

dotenv.load_dotenv()
email = os.getenv("EMAIL")
senha = os.getenv("SENHA")
app_key = os.getenv("APP_KEY")
print(email)

In [ ]:
#Fazendo a conexão com BetFair
trading = betfairlightweight.APIClient(email, senha, app_key=app_key, cert_files=('/content/FutBetBot.crt','/content/FutBetBot.pem'))
trading.login()

In [ ]:
# Definindo os filtros para futebol
filtros_competicoes = betfairlightweight.filters.market_filter(
        event_type_ids=['1'],  # ID para Futebol
        market_start_time={'to': (datetime.utcnow() + timedelta(days=1)).strftime("%Y-%m-%dT%TZ")})#, competition_ids = id_list) # Competições

#Dados jogos de hoje e amanhã
jogos_futebol = trading.betting.list_events(
    filter=filtros_competicoes
)

In [ ]:
#Mostar lista com nomes dos jogos

lista_jogos = []
for partida in jogos_futebol:
    jogo = partida.event.name
    lista_jogos.append(jogo)

lista_jogos

In [ ]:
#Cria um DataFrame pandas com os dados dos jogos do dia

jogos_do_dia = pd.DataFrame({
    'NomeEvento': [partida.event.name for partida in jogos_futebol],
    'IDEvento': [partida.event.id for partida in jogos_futebol],
    'CodPais': [partida.event.country_code for partida in jogos_futebol],
    'TimeZone': [partida.event.time_zone for partida in jogos_futebol],
    'DataAbertura': [partida.event.open_date for partida in jogos_futebol],
    'TotalMercados': [partida.market_count for partida in jogos_futebol],
    'DataLocal': [partida.event.open_date.replace(tzinfo=timezone.utc).astimezone(tz=None)
                        for partida in jogos_futebol]
})

jogos_do_dia